In [1]:
from py2neo import Graph, Node, Relationship

In [2]:
g=Graph("bolt://localhost:7687",password="user")

#  Q1-RETRIEVE THE BARGAIN TRANSFERS 

In [12]:
q1 = '''MATCH (clubTo:Club)<-[r1:teamTo]-(transfer:Transfer)-[r:playerInvolved]->(player:Player)
WHERE toInteger(transfer.fee) < toInteger(r.MarketValue) and toInteger(transfer.fee) <> 0
RETURN player.name, toInteger(transfer.fee),toInteger(r.MarketValue), clubTo.name, toInteger(transfer.fee)* 1.0/toInteger(r.MarketValue) as div
ORDER BY div ASC
'''

In [13]:
g.run(q1).to_table()

player.name,toInteger(transfer.fee),toInteger(r.MarketValue),clubTo.name,div
Bernd Leno,502452,5000000,Bay. Leverkusen,0.1004904
Iago Falque,1004904,8000000,AS Roma,0.125613
Tin Jedvaj,1004904,6500000,Bay. Leverkusen,0.1546006153846154
Pizzi,1507349,7500000,Atl�tico Madrid,0.20097986666666667
Mapou Yanga-Mbiwa,1261707,6000000,AS Roma,0.2102845
Jos� Antonio Reyes,3517148,15000000,Sevilla FC,0.23447653333333332
Emmanuel Adebayor,6431357,25000000,Spurs,0.25725428
Maicon,3773956,14500000,Man City,0.2602728275862069
Fabio Borini,1261707,4800000,AS Roma,0.262855625
Nigel de Jong,3517148,13000000,AC Milan,0.27054984615384614


# Q2-Top3 buyer clubs per season. Note we used the extra edge created for the algorithms.

In [25]:
q2 = '''
MATCH (c2:Club)-[tr:transfer]->(c1:Club)<-[tto:teamTo]-(t:Transfer)-[ts:transferSeason]->(s:Season)
WITH s.year AS season, c1.name AS clubName, COUNT(DISTINCT ts) AS numberOfBuys
ORDER BY season, numberOfBuys DESC
WITH season, collect(clubName) AS clubList, collect(numberOfBuys) AS buys
UNWIND RANGE(0, 2) AS i
WITH season, [clubList[i], buys[i]] AS output
WITH season, COLLECT(output) AS output_list
RETURN season, output_list
ORDER BY season
'''


In [26]:
g.run(q2).to_table()

season,output_list
2009-2010,"[['Shakhtar D.', 2], ['Brescia', 1], ['Birmingham', 1]]"
2010-2011,"[['Genoa', 10], ['Man City', 7], ['Real Madrid', 6]]"
2011-2012,"[['Juventus', 8], ['AS Roma', 8], ['Paris SG', 7]]"
2012-2013,"[['Chelsea', 7], ['Juventus', 7], ['Man Utd', 6]]"
2013-2014,"[['SSC Napoli', 10], ['Monaco', 8], ['Spurs', 7]]"
2014-2015,"[['Liverpool', 8], ['Southampton', 8], ['Atl\ufffdtico Madrid', 7]]"
2015-2016,"[['Valencia CF', 8], ['Watford', 8], ['Monaco', 7]]"
2016-2017,"[['Inter', 9], ['Bor. Dortmund', 8], ['AS Roma', 7]]"
2017-2018,"[['AC Milan', 7], ['Juventus', 7], ['Everton', 5]]"


# Q3-Top transferred players:

In [16]:
q3 = '''MATCH (t:Transfer)-[ts:playerInvolved]->(p:Player)
RETURN p.name, COUNT(t) AS numberOfTransfers
ORDER BY numberOfTransfers DESC
'''

In [17]:
g.run(q3).to_table()

p.name,numberOfTransfers
Ciro Immobile,6
Fernando,5
Mattia Destro,5
Roberto,5
Andr� Sch�rrle,4
Christopher Samba,4
Danilo,4
Dimitri Payet,4
Dorlan Pab�n,4
Fabio Borini,4


# Q4-Most expensive positions:

In [18]:
q4 = '''
MATCH (transfer:Transfer)-[r:playerInvolved]->(player:Player) -[r1:playsAs]->(role:Role) 
RETURN role.position, avg(toInteger(transfer.fee)) as tf
ORDER BY tf DESC
'''

In [19]:
g.run(q4).to_table()

role.position,tf
Left Wing,15066724.106250001
Central Midfield,13239977.783251233
Right Wing,13039366.523178805
Attacking Midfield,12441405.285714285
Left Midfield,12314007.071428573
Centre-Forward,12257117.116704805
Centre-Back,11165594.385826776
Defensive Midfield,10900191.848101266
Left-Back,10570765.292134833
Right-Back,10536497.677419357


# RECCOMENDER SYSTEM: RETRIEVE ALL THE BARGAIN TRANSFERS FROM LESS THAN 3 AGES AND WITH PLAYERS YOUNGER THAN AN INPUT AGE. WE SUGGEST TO INPUT 21 OR 22.

In [21]:
x = (input())
queryRec = '''
MATCH (season:Season)<-[r1:transferSeason]-(transfer:Transfer)-[r:playerInvolved]->(player:Player)
WHERE toInteger(transfer.fee) < toInteger(r.MarketValue) and toInteger(transfer.fee) <> 0 and toInteger(r.Age)<'''+ x +''' and 2018 - toInteger(right(season.year,4)) <4 
RETURN player.name,r.Age,season.year, toInteger(transfer.fee),toInteger(r.MarketValue),  toInteger(transfer.fee)* 1.0/toInteger(r.MarketValue) as div
ORDER BY div ASC
'''

22


In [22]:
g.run(queryRec).to_table()

player.name,r.Age,season.year,toInteger(transfer.fee),toInteger(r.MarketValue),div
Tin Jedvaj,18,2014-2015,1004904,6500000,0.1546006153846154
Serge Gnabry,21,2017-2018,8039196,15000000,0.5359464
Domenico Berardi,20,2015-2016,10048995,17500000,0.5742282857142857
Massimo Bruno,20,2014-2015,5024497,8000000,0.628062125
D�ria,19,2014-2015,5024497,8000000,0.628062125
Luciano Vietto,20,2014-2015,5526947,8500000,0.6502290588235294
Daniele Rugani,20,2014-2015,5024497,7500000,0.6699329333333334
Josip Drmic,21,2014-2015,6833317,10000000,0.6833317
Michy Batshuayi,20,2014-2015,6029397,8000000,0.753674625
Fabinho,21,2015-2016,6029397,8000000,0.753674625
